In [0]:
# 00_materialize_rules_as_py (generic: recursive)
import os

# repo root
nb_path = (
    dbutils.notebook.entry_point.getDbutils()
    .notebook()
    .getContext()
    .notebookPath()
    .get()
)
repo_ws_root = "/".join(nb_path.split("/")[:4])   # /Repos/<user>/<repo>
repo_fs_root = f"/Workspace{repo_ws_root}"

out_fs_root = f"{repo_fs_root}/materialized_py"
out_ws_root = f"{repo_ws_root}/materialized_py"


def ensure_dir(p: str) -> None:
    os.makedirs(p, exist_ok=True)


def write_file(path: str, text: str) -> None:
    ensure_dir(os.path.dirname(path))
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)


def strip_notebook_markers(text: str) -> str:
    # Remove Databricks notebook markers so output is "plain .py"
    out = []
    for ln in text.splitlines():
        if ln.startswith("# Databricks notebook source"):
            continue
        if ln.startswith("# COMMAND ----------"):
            continue
        out.append(ln)
    return "\n".join(out).strip() + "\n"


def ensure_init_py(dst_dir: str) -> None:
    init_path = os.path.join(dst_dir, "__init__.py")
    if not os.path.exists(init_path):
        write_file(init_path, "")


def materialize_tree(rel_src_root: str) -> None:
    src_root = f"{repo_fs_root}/{rel_src_root}"
    dst_root = f"{out_fs_root}/{rel_src_root}"
    ensure_dir(dst_root)

    for cur_dir, _, files in os.walk(src_root):
        rel = os.path.relpath(cur_dir, src_root)  # "." or subdir
        dst_dir = os.path.normpath(os.path.join(dst_root, rel))
        ensure_dir(dst_dir)
        ensure_init_py(dst_dir)

        for fname in sorted(files):
            if fname.startswith("_"):
                continue
            if not fname.endswith(".py"):
                continue

            src_path = os.path.join(cur_dir, fname)
            dst_path = os.path.join(dst_dir, fname)

            with open(src_path, "r", encoding="utf-8") as f:
                raw = f.read()

            clean = strip_notebook_markers(raw)
            write_file(dst_path, clean)


# package root init
ensure_dir(out_fs_root)
ensure_init_py(out_fs_root)

# detections 전체 + lib 전체
materialize_tree("base/detections")  # binary/behavioral/custom 모두 포함
materialize_tree("lib")

print("OK: materialized to", out_ws_root)
